### Import

In [1]:
import pandas as pd
import numpy as np

### Data Load

In [4]:
train = pd.read_csv('./RAWDATA/train.csv').drop(columns=['ID'])
test = pd.read_csv('./RAWDATA/test.csv').drop(columns=['ID'])

In [5]:
X_train = train.drop('임신 성공 여부', axis=1)
y_train = train['임신 성공 여부']

In [6]:
# '''
#     TODO : y_train 값 분포 비율
# '''
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(6,4))
# sns.histplot(y_train, kde=True, bins=30, color='skyblue')
# plt.title('y_train Distribution')
# plt.xlabel('y_train')
# plt.ylabel('Frequency')
# plt.show()

### Data Pre-processing

In [7]:
'''
    TODO : 수치형 데이터 구분
'''
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
] # 20

In [8]:
'''
    TODO : 수치형 데이터 null 값 확인
'''
for col in numeric_columns:
    print(X_train[col].isnull().sum())
numeric_null_columns = [ col for col in numeric_columns if X_train[col].isnull().sum() != 0 ]
print(numeric_null_columns)

246981
6291
6291
6291
6291
6291
6291
6291
6291
6291
6291
6291
6291
6291
6291
57488
254915
53735
43566
215982
['임신 시도 또는 마지막 임신 경과 연수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']


In [9]:
'''
    TODO : 수치형 데이터 결측치 처리 (drop)
'''
numeric_null_ratio = X_train[numeric_null_columns].isnull().mean().sort_values(ascending=False)
print(numeric_null_ratio)

drop_columns = numeric_null_ratio[numeric_null_ratio >= 0.8].index
X_train = X_train.drop(columns=drop_columns)
test = test.drop(columns=drop_columns)

numeric_columns = list(set(numeric_columns) - set(drop_columns))
print(f"삭제된 칼럼: {list(drop_columns)}")
print(f"남은 numeric_columns: {numeric_columns}")

난자 해동 경과일                0.994398
임신 시도 또는 마지막 임신 경과 연수    0.963449
배아 해동 경과일                0.842525
난자 채취 경과일                0.224255
난자 혼합 경과일                0.209615
배아 이식 경과일                0.169947
미세주입에서 생성된 배아 수          0.024541
혼합된 난자 수                 0.024541
미세주입된 난자 수               0.024541
기증자 정자와 혼합된 난자 수         0.024541
파트너 정자와 혼합된 난자 수         0.024541
저장된 신선 난자 수              0.024541
이식된 배아 수                 0.024541
총 생성 배아 수                0.024541
해동 난자 수                  0.024541
해동된 배아 수                 0.024541
미세주입 후 저장된 배아 수          0.024541
저장된 배아 수                 0.024541
미세주입 배아 이식 수             0.024541
수집된 신선 난자 수              0.024541
dtype: float64
삭제된 칼럼: ['난자 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수', '배아 해동 경과일']
남은 numeric_columns: ['파트너 정자와 혼합된 난자 수', '저장된 배아 수', '난자 혼합 경과일', '혼합된 난자 수', '수집된 신선 난자 수', '미세주입에서 생성된 배아 수', '해동된 배아 수', '기증자 정자와 혼합된 난자 수', '이식된 배아 수', '미세주입된 난자 수', '난자 채취 경과일', '총 생성 배아 수', '배아 이식 경과일', '해동 난자 수', '미세주입 배아 이식 수', '저장된 신선 난

In [10]:
'''
    TODO : 수치형 데이터 결측치 처리 (filled) - KNN imputer
'''
import joblib 
from sklearn.impute import KNNImputer

imputer_cache_path = "knn_imputed.pkl"

try:
    X_train_imputed, test_imputed = joblib.load(imputer_cache_path)
    print("Loaded cached KNN imputed data.")
except FileNotFoundError:
    numeric_filled_columns = numeric_null_ratio[numeric_null_ratio < 0.8].index  # 결측치 비율 80% 미만 컬럼 선택
    
    knn_imputer = KNNImputer(n_neighbors=6)
    
    X_train_imputed = X_train.copy()
    test_imputed = test.copy()
    
    X_train_imputed[numeric_filled_columns] = knn_imputer.fit_transform(X_train[numeric_filled_columns])
    print("X_train finish")
    test_imputed[numeric_filled_columns] = knn_imputer.transform(test[numeric_filled_columns])
    print("All columns imputed.")
    
    joblib.dump((X_train_imputed, test_imputed), imputer_cache_path)
    print("Saved KNN imputed data.")


KeyboardInterrupt: 

In [9]:
'''
    TODO : 순서가 있는 범주형 데이터  구분
'''
categorical_columns_level = [
    "시술 당시 나이",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 기증자 나이",
    "정자 기증자 나이",
] # 13

In [10]:
'''
    TODO : 순서가 있는 범주형 데이터 null 값 확인 
'''
for col in categorical_columns_level:
    print(X_train_imputed[col].isnull().sum())
# 결측지 없어서 결측지 처리 따로 안함

0
0
0
0
0
0
0
0
0
0
0
0
0


In [11]:
# '''
#     TODO : 순서가 있는 범주형 데이터 
# '''
# for col in categorical_columns_level:
#     X_train_imputed[col] = X_train_imputed[col].astype(str)
#     test_imputed[col] = test_imputed[col].astype(str)

In [12]:
'''
    TODO : 순서가 없는 범주형 데이터 구분
'''
categorical_columns = [
    "시술 시기 코드",
    "시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "난자 출처",
    "정자 출처",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    "특정 시술 유형"
] #34

In [13]:
'''
    TODO : 순서가 없는 범주형 데이터 null 값 확인
'''
categorical_null_columns = []
print(X_train_imputed[categorical_columns].isnull().sum())
categorical_null_columns = [ col for col in categorical_columns if X_train_imputed[col].isnull().sum() != 0 ]
print(categorical_null_columns)

시술 시기 코드                   0
시술 유형                      0
배란 자극 여부                   0
배란 유도 유형                   0
단일 배아 이식 여부             6291
착상 전 유전 검사 사용 여부      253633
착상 전 유전 진단 사용 여부        6291
남성 주 불임 원인                 0
남성 부 불임 원인                 0
여성 주 불임 원인                 0
여성 부 불임 원인                 0
부부 주 불임 원인                 0
부부 부 불임 원인                 0
불명확 불임 원인                  0
불임 원인 - 난관 질환              0
불임 원인 - 남성 요인              0
불임 원인 - 배란 장애              0
불임 원인 - 여성 요인              0
불임 원인 - 자궁경부 문제            0
불임 원인 - 자궁내막증              0
불임 원인 - 정자 농도              0
불임 원인 - 정자 면역학적 요인         0
불임 원인 - 정자 운동성             0
불임 원인 - 정자 형태              0
배아 생성 주요 이유             6291
난자 출처                      0
정자 출처                      0
동결 배아 사용 여부             6291
신선 배아 사용 여부             6291
기증 배아 사용 여부             6291
대리모 여부                  6291
PGD 시술 여부             254172
PGS 시술 여부             254422
특정 시술 유형                   2
dtype: int64
[

In [14]:
'''
    TODO : 순서가 없는 범주형 데이터 결측치 처리 (drop)
'''
categorical_null_ratio = X_train_imputed[categorical_null_columns].isnull().mean().sort_values(ascending=False)
print(categorical_null_ratio)

drop_columns = categorical_null_ratio[categorical_null_ratio >= 0.9].index
X_train_imputed = X_train_imputed.drop(columns=drop_columns)
test_imputed = test_imputed.drop(columns=drop_columns)

categorical_columns = list(set(categorical_columns) - set(drop_columns))
print(f"삭제된 칼럼: {list(drop_columns)}")
print(f"남은 categorical_columns: {categorical_columns}")
print(len(categorical_columns))

PGS 시술 여부           0.992475
PGD 시술 여부           0.991500
착상 전 유전 검사 사용 여부    0.989397
단일 배아 이식 여부         0.024541
착상 전 유전 진단 사용 여부    0.024541
배아 생성 주요 이유         0.024541
동결 배아 사용 여부         0.024541
신선 배아 사용 여부         0.024541
기증 배아 사용 여부         0.024541
대리모 여부              0.024541
특정 시술 유형            0.000008
dtype: float64
삭제된 칼럼: ['PGS 시술 여부', 'PGD 시술 여부', '착상 전 유전 검사 사용 여부']
남은 categorical_columns: ['불임 원인 - 정자 면역학적 요인', '불임 원인 - 여성 요인', '배아 생성 주요 이유', '불임 원인 - 정자 농도', '동결 배아 사용 여부', '불임 원인 - 자궁경부 문제', '시술 유형', '신선 배아 사용 여부', '부부 부 불임 원인', '특정 시술 유형', '남성 주 불임 원인', '난자 출처', '여성 주 불임 원인', '불임 원인 - 남성 요인', '시술 시기 코드', '부부 주 불임 원인', '단일 배아 이식 여부', '불임 원인 - 정자 형태', '착상 전 유전 진단 사용 여부', '배란 자극 여부', '불임 원인 - 배란 장애', '기증 배아 사용 여부', '불임 원인 - 난관 질환', '불임 원인 - 정자 운동성', '불명확 불임 원인', '대리모 여부', '배란 유도 유형', '남성 부 불임 원인', '여성 부 불임 원인', '정자 출처', '불임 원인 - 자궁내막증']
31


In [15]:
'''
    TODO : 순서가 없는 범주형 데이터 결측치 처리 (filled)
'''
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

X_train_imputed[categorical_columns] = imputer.fit_transform(X_train_imputed[categorical_columns])
test_imputed[categorical_columns] = imputer.transform(test_imputed[categorical_columns])

print("'Missing'으로 결측치 채움 완료")

'Missing'으로 결측치 채움 완료


In [16]:
'''
    TODO : 범주형 데이터 인코딩 준비
'''
categorical_columns = list(set(categorical_columns) | set(categorical_columns_level))
print(len(categorical_columns))

for col in categorical_columns:
    X_train_imputed[col] = X_train_imputed[col].astype(str)
    test_imputed[col] = test_imputed[col].astype(str)

44


In [17]:
'''
    TODO : 범주형 데이터 인코딩 - Ordinal Encoder
'''
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 카테고리형 변수 인코딩
X_train_encoded = X_train_imputed.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X_train_imputed[categorical_columns])

test_encoded = test_imputed.copy()
test_encoded[categorical_columns] = ordinal_encoder.transform(test_imputed[categorical_columns])


In [18]:
# '''
#     TODO : PCA 적용 (수치형 데이터) -> 성능에 별 차지를 주지는 않군
# '''
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# X_train_numeric = X_train[numeric_columns]
# test_numeric = test[numeric_columns]

# # 데이터 스케일링 (PCA는 거리 기반 기법이므로 반드시 표준화 필요)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_numeric)
# test_scaled = scaler.transform(test_numeric)

# # PCA 적용 (설정: 분산을 95% 유지하는 주성분 개수 자동 선택)
# pca = PCA(n_components=0.95, random_state=42)
# X_train_pca = pca.fit_transform(X_train_scaled)
# test_pca = pca.transform(test_scaled)

# # PCA 결과를 데이터프레임으로 변환
# X_train_pca = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])])
# test_pca = pd.DataFrame(test_pca, columns=[f'PC{i+1}' for i in range(test_pca.shape[1])])

# # PCA 설명된 분산 비율 확인
# explained_variance = pca.explained_variance_ratio_
# cumulative_variance = explained_variance.cumsum()
# num_components = len(cumulative_variance)

# print(f"PCA :  사용된 주성분 개수: {num_components}")
# print(f"설명된 분산 비율 (누적): {cumulative_variance[-1]:.4f}")

# # 기존 데이터에서 숫자형 부분을 PCA 결과로 교체
# X_train.update(X_train_pca)
# test.update(test_pca)


### Train

In [1252]:
# from catboost import CatBoostClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, roc_auc_score

# # 데이터 80:20으로 분할
# X_train_split, X_valid, y_train_split, y_valid = train_test_split(
#     X_train_encoded, y_train, test_size=0.2, random_state=42
# )

# # CatBoost 모델 학습
# catboost_model = CatBoostClassifier(l2_leaf_reg=5, random_state=42, verbose=100)
# catboost_model.fit(X_train_split, y_train_split)

# # 검증 데이터에서 예측 수행
# y_pred_valid = catboost_model.predict(X_valid)
# y_pred_valid_proba = catboost_model.predict_proba(X_valid)[:, 1]  # 양성 클래스 확률

# # 검증 데이터 평가
# accuracy_valid = accuracy_score(y_valid, y_pred_valid)
# roc_auc_valid = roc_auc_score(y_valid, y_pred_valid_proba)

# print(f"Validation Accuracy: {accuracy_valid:.4f}")
# print(f"Validation ROC-AUC Score: {roc_auc_valid:.4f}")

# # 학습 데이터에서도 평가 (과적합 여부 확인)
# y_pred_train = catboost_model.predict(X_train_split)
# y_pred_train_proba = catboost_model.predict_proba(X_train_split)[:, 1]

# accuracy_train = accuracy_score(y_train_split, y_pred_train)
# roc_auc_train = roc_auc_score(y_train_split, y_pred_train_proba)

# print(f"Train Accuracy: {accuracy_train:.4f}")
# print(f"Train ROC-AUC Score: {roc_auc_train:.4f}")


In [19]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# CatBoost 모델 학습
catboost_model = CatBoostClassifier(random_state=42, verbose=100)
catboost_model.fit(X_train_encoded, y_train)

# 학습 데이터에서 예측 수행
y_pred_train = catboost_model.predict(X_train_encoded)
y_pred_train_proba = catboost_model.predict_proba(X_train_encoded)[:, 1]

# 학습 데이터 평가
accuracy_train = accuracy_score(y_train, y_pred_train)
roc_auc_train = roc_auc_score(y_train, y_pred_train_proba)

print(f"Train Accuracy: {accuracy_train:.4f}")
print(f"Train ROC-AUC Score: {roc_auc_train:.4f}")

Learning rate set to 0.110029
0:	learn: 0.6262906	total: 98ms	remaining: 1m 37s
100:	learn: 0.4862047	total: 4.82s	remaining: 42.9s
200:	learn: 0.4829905	total: 8.74s	remaining: 34.8s
300:	learn: 0.4805997	total: 12.2s	remaining: 28.4s
400:	learn: 0.4786102	total: 15.7s	remaining: 23.5s
500:	learn: 0.4769000	total: 19.3s	remaining: 19.2s
600:	learn: 0.4752877	total: 23.1s	remaining: 15.3s
700:	learn: 0.4738480	total: 26.8s	remaining: 11.4s
800:	learn: 0.4723504	total: 30.4s	remaining: 7.54s
900:	learn: 0.4710740	total: 33.8s	remaining: 3.72s
999:	learn: 0.4699630	total: 37.3s	remaining: 0us
Train Accuracy: 0.7613
Train ROC-AUC Score: 0.7704


In [1254]:
# '''
#  TODO : 교차 검증
# '''
# from sklearn.model_selection import cross_val_score, StratifiedKFold

# # 교차 검증 설정 (Stratified K-Fold 사용)
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # 교차 검증 수행
# cv_scores = cross_val_score(catboost_model, X_train, y_train, cv=cv, scoring='accuracy')

# # 결과 출력
# print(f"Cross-validation Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

In [1255]:
# 특성 중요도 가져오기
feature_importance = catboost_model.get_feature_importance()
feature_names = X_train_encoded.columns

# 중요도 시각화
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df.head(10)) 
print(importance_df.tail(10))

        Feature  Importance
39     이식된 배아 수   81.998646
1      시술 당시 나이    4.240211
60    배아 이식 경과일    2.649777
41     저장된 배아 수    1.807420
36    총 생성 배아 수    1.074758
45  수집된 신선 난자 수    0.729901
0      시술 시기 코드    0.510227
52    난자 기증자 나이    0.510068
50        난자 출처    0.502550
5      배란 유도 유형    0.409640
               Feature  Importance
54         동결 배아 사용 여부    0.008488
9           남성 부 불임 원인    0.006133
57              대리모 여부    0.005228
23      불임 원인 - 정자 운동성    0.005145
21       불임 원인 - 정자 농도    0.005003
19     불임 원인 - 자궁경부 문제    0.001126
22  불임 원인 - 정자 면역학적 요인    0.000010
18       불임 원인 - 여성 요인    0.000000
58           난자 채취 경과일    0.000000
24       불임 원인 - 정자 형태    0.000000


In [1256]:
# '''
#     TODO : ROC-AUC 곡선을 그리는 함수
#          :param y_true: 실제 타겟 값 (0 또는 1)
#          :param y_pred_proba: 모델이 예측한 양성 클래스(1)의 확률값
# '''
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, auc, roc_auc_score

# def plot_roc_auc(y_true, y_pred_proba):
#     # ROC 커브 계산
#     fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
#     roc_auc = auc(fpr, tpr)  # AUC 값 계산

#     # 그래프 그리기
#     plt.figure(figsize=(6, 4))
#     plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
#     plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # 랜덤 분류기 기준선
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate (FPR)')
#     plt.ylabel('True Positive Rate (TPR)')
#     plt.title('Receiver Operating Characteristic (ROC) Curve')
#     plt.legend(loc='lower right')
#     plt.grid()
#     plt.show()

# # 사용 예시
# # 모델 예측 확률 (pred_proba 사용)
# y_pred_proba = catboost_model.predict_proba(X_train_encoded)[:, 1]
# print(f"ROC-AUC Score : {roc_auc_score(y_train, y_pred_proba)}")

# # # ROC-AUC 그래프 출력
# # plot_roc_auc(y_train, y_pred_proba)
# # y_pred_proba = stacking_model.predict_proba(X_train)[:, 1]

# # ROC-AUC 그래프 출력
# plot_roc_auc(y_train, y_pred_proba)

### Predict

In [20]:
pred_proba = catboost_model.predict_proba(test_encoded)[:, 1]


### Submission

In [21]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [22]:
sample_submission.to_csv('./baseline_submit.csv', index=False)